# 0.0 Introduction
This code attempts to solve the challenge hosted on DrivenData.org The challenge can be accessed via https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table

A walk-through of this code is available at: 

This is the second of two parts that attempts to solve the challenge. The first part can be found at the following links below:

Github: https://github.com/AntonioStark/Taarifa-Water-Pumps/blob/master/Taarifa%20Water%20Pumps%20pt.1%20Data%20Preprocessing%20EDA.ipynb

Medium: https://medium.com/@antoniostark/50ba6ea65c0a

# 1.0 Problem Description
The challenge is to create a model that will predict the condition of a particular water pump ("waterpoint") given its many attributes

In [1]:
# general package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 2.1 Data preparation
We've finally managed to get our data preprocessed! Let's run it through some models and see if it gives us any significant results!

While there are many types of classifiers we can use, they are generally put into thse three families: nearest neighbors, decision trees, and support vector machines. We're going to look at one example model from each family of models.
